In [3]:
import json
import re
from typing import Dict, Any, List
import sys
sys.path.append(r"D:\Data\Legal-Retrieval\lexiops-copilot")

In [11]:
essential_tools = [
    "kubectl",          # Most flexible and reliable
    "k8s_get",         # Get resources
    "k8s_logs",        # View logs
    "k8s_describe",    # Describe resources
    "k8s_create",      # Create resources
    "k8s_delete",      # Delete resources
    "k8s_scale",       # Scale resources
    "k8s_apply",       # Apply YAML
    "k8s_rollout_status", # Check deployments
    "k8s_events",      # View events
    "k8s_top_pods",    # Resource usage
    "helm"             # Helm operations
]

In [12]:
def parse_mcp_tool(raw_tool_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Chuyển đổi cấu trúc tool thô từ MCP server thành định dạng JSON Schema
    thân thiện với LLM, giàu thông tin hơn.
    (Hàm này giữ nguyên như trước)
    """
    tool_name = raw_tool_data.get("name", "unknown_tool")
    raw_description = raw_tool_data.get("description", "")
    input_schema = raw_tool_data.get("inputSchema", {})
    
    main_description = raw_description.split("\n\n:param")[0].strip()

    cleaned_properties = {}
    required_params = input_schema.get("required", [])
    
    param_descriptions = {}
    matches = re.findall(r":param\s+([^:]+):\s+(.+)", raw_description)
    for match in matches:
        param_name, desc = match
        param_descriptions[param_name.strip()] = desc.strip()

    for param_name, param_info in input_schema.get("properties", {}).items():
        param_type = "string" # Mặc định
        if param_info.get("default") is False or param_info.get("default") is True:
            param_type = "boolean"
        # Bạn có thể thêm các logic suy luận type khác ở đây
        
        cleaned_properties[param_name] = {
            "type": param_type,
            "description": param_descriptions.get(param_name, param_info.get("title", "")),
        }

    final_tool_definition = {
        "name": tool_name,
        "description": main_description,
        "parameters": {
            "type": "object",
            "properties": cleaned_properties,
            "required": required_params
        }
    }
    
    return final_tool_definition

In [15]:
def process_tool_list(input_filepath: str, output_filepath: str):
    """
    Đọc một file JSON chứa danh sách các tool thô, xử lý chúng,
    và lưu kết quả đã được tối ưu vào một file JSON mới.
    """
    try:
        with open(input_filepath, "r", encoding="utf-8") as f:
            raw_tools_list = json.load(f)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file '{input_filepath}'")
        return
    except json.JSONDecodeError:
        print(f"Lỗi: File '{input_filepath}' không phải là file JSON hợp lệ.")
        return

    cleaned_tools_list = []
    print(f"Đang xử lý {len(raw_tools_list)} tool từ '{input_filepath}'...")

    for raw_tool in raw_tools_list:
        if raw_tool.get("name", "unknown_tool") not in essential_tools:
            continue
        cleaned_tool = parse_mcp_tool(raw_tool)
        cleaned_tools_list.append(cleaned_tool)
    
    with open(output_filepath, "w", encoding="utf-8") as f:
        json.dump(cleaned_tools_list, f, indent=2, ensure_ascii=False)
        
    print(f"✅ Xử lý thành công! Đã lưu {len(cleaned_tools_list)} tool đã được tối ưu vào '{output_filepath}'.")

In [16]:
raw_tools_data = []
input_file = r"D:\Data\Legal-Retrieval\lexiops-copilot\generated\mcp_tools\mcp_tools_full.json"
with open(input_file, "r", encoding="utf-8") as f:
    raw_tools_data = json.load(f)


In [17]:
process_tool_list(input_filepath=input_file, output_filepath="cleaned_mcp_tools.json")

Đang xử lý 39 tool từ 'D:\Data\Legal-Retrieval\lexiops-copilot\generated\mcp_tools\mcp_tools_full.json'...
✅ Xử lý thành công! Đã lưu 12 tool đã được tối ưu vào 'cleaned_mcp_tools.json'.
